<a href="https://colab.research.google.com/github/tubagokhan/RegNLP2025/blob/main/RIRAGEvaluationMetricFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1BB21XL5geQ0Uw6gk0_uRFk6tthPQ0vSh
!gdown 1BGb3EYmhmTMVx3qTihCBpUrJ3WO2lM8H
!gdown 10NPnxgrv-Pg-RErN-zYhmjbQ9EMZZkG8
!gdown 10KyGhlmjeCtqiNLfNDGYjR2lnsv_2_Rc

Downloading...
From: https://drive.google.com/uc?id=1BB21XL5geQ0Uw6gk0_uRFk6tthPQ0vSh
To: /content/ObligationClassificationDataset.json
100% 545k/545k [00:00<00:00, 143MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BGb3EYmhmTMVx3qTihCBpUrJ3WO2lM8H
To: /content/selected_test_samples.json
100% 135k/135k [00:00<00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=10NPnxgrv-Pg-RErN-zYhmjbQ9EMZZkG8
To: /content/retrieval_results.passage_only_bm25_answers.json
100% 3.00M/3.00M [00:00<00:00, 223MB/s]
Downloading...
From: https://drive.google.com/uc?id=10KyGhlmjeCtqiNLfNDGYjR2lnsv_2_Rc
To: /content/retrieval_results.rank_fusion_bm25_answers.json
100% 3.18M/3.18M [00:00<00:00, 157MB/s]


In [2]:
import json
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check if CUDA is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Load and preprocess the data
json_path = "./ObligationClassificationDataset.json"
with open(json_path, 'r') as file:
    data = json.load(file)

texts = [item['Text'] for item in data]
labels = [1 if item['Obligation'] else 0 for item in data]  # Converting True/False to 1/0

# Step 2: Tokenization using LegalBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

class ObligationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_dataset = ObligationDataset(X_train, y_train, tokenizer)
val_dataset = ObligationDataset(X_val, y_val, tokenizer)

# Step 3: Fine-tuning LegalBERT for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=2)
model.to(device)  # Move model to the GPU

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Step 4: Train the model
trainer.train()

# Step 5: Evaluate the model
trainer.evaluate()

# Step 6: Save the model and tokenizer for future use
model.save_pretrained('./obligation-classifier-legalbert')
tokenizer.save_pretrained('./obligation-classifier-legalbert')

print("Model fine-tuning and evaluation completed.")


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.502700,0.327538,0.902174,0.924370,0.867508,0.989209
2,0.069200,0.045332,0.991304,0.992754,1.000000,0.985612
3,0.073200,0.047527,0.989130,0.991055,0.985765,0.996403
4,0.001200,0.054512,0.989130,0.990958,0.996364,0.985612
5,0.046400,0.125565,0.980435,0.983607,0.996310,0.971223


Model fine-tuning and evaluation completed.


In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 320.6 kB/s eta 0:00:00


In [4]:
import json
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from nltk.tokenize import sent_tokenize as sent_tokenize_uncached
import nltk
from functools import cache
import tqdm

nltk.download('punkt')

# Set up device for torch operations
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the tokenizer and model for obligation detection
model_name = './obligation-classifier-legalbert'
obligation_tokenizer = AutoTokenizer.from_pretrained(model_name)
obligation_model = AutoModelForSequenceClassification.from_pretrained(model_name)
obligation_model.to(device)
obligation_model.eval()

# Load NLI model and tokenizer for obligation coverage using Microsoft's model
coverage_nli_model = pipeline("text-classification", model="microsoft/deberta-large-mnli", device=device)

# Load NLI model and tokenizer for entailment and contradiction checks
nli_tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-xsmall')
nli_model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-xsmall')
nli_model.to(device)
nli_model.eval()

# Define a cached version of sentence tokenization
@cache
def sent_tokenize(passage: str):
  return sent_tokenize_uncached(passage)

def softmax(logits):
    e_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return e_logits / np.sum(e_logits, axis=1, keepdims=True)

def get_nli_probabilities(premises, hypotheses):
    features = tokenizer(premises, hypotheses, padding=True, truncation=True, return_tensors="pt").to("cuda")
    nli_model.eval()
    with torch.no_grad():
        logits = nli_model(**features).logits.cpu().numpy()
    probabilities = softmax(logits)
    return probabilities

def get_nli_matrix(passages, answers):
    print(f"{len(passages)} passages and {len(answers)} answers.")
    entailment_matrix = np.zeros((len(passages), len(answers)))
    contradiction_matrix = np.zeros((len(passages), len(answers)))

    batch_size = 16
    for i, pas in enumerate(tqdm.tqdm(passages)):
      for b in range(0, len(answers), batch_size):
        e = b + batch_size
        probs = get_nli_probabilities([pas] * len(answers[b:e]), answers[b:e])  # Get NLI probabilities
        entailment_matrix[i, b:e] = probs[:, 1]
        contradiction_matrix[i, b:e] = probs[:, 0]
    return entailment_matrix, contradiction_matrix

def calculate_scores_from_matrix(nli_matrix, score_type='entailment'):
    if nli_matrix.size == 0:
        print("Warning: NLI matrix is empty. Returning default score of 0.")
        return 0.0  # or some other default score or handling as appropriate for your use case

    if score_type == 'entailment':
        reduced_vector = np.max(nli_matrix, axis=0)
    elif score_type == 'contradiction':
        reduced_vector = np.max(nli_matrix, axis=0)
    score = np.round(np.mean(reduced_vector), 5)
    return score

def calculate_obligation_coverage_score(passages, answers):
    obligation_sentences_source = [sent for passage in passages for sent in sent_tokenize(passage)]
    obligation_sentences_answer = [sent for answer in answers for sent in sent_tokenize(answer)]
    covered_count = 0

    for obligation in obligation_sentences_source:
        obligation_covered = False
        for answer_sentence in obligation_sentences_answer:
            nli_result = coverage_nli_model(f"{answer_sentence} [SEP] {obligation}")
            if nli_result[0]['label'].lower() == 'entailment' and nli_result[0]['score'] > 0.7:
                covered_count += 1
                obligation_covered = True
                break


    coverage_score = covered_count / len(obligation_sentences_source) if obligation_sentences_source else 0
    return coverage_score

def calculate_final_composite_score(passages, answers):
    passage_sentences = [sent for passage in passages for sent in sent_tokenize(passage)]
    answer_sentences = [sent for answer in answers for sent in sent_tokenize(answer)]
    entailment_matrix, contradiction_matrix = get_nli_matrix(passage_sentences, answer_sentences)
    entailment_score = calculate_scores_from_matrix(entailment_matrix, 'entailment')
    contradiction_score = calculate_scores_from_matrix(contradiction_matrix, 'contradiction')
    obligation_coverage_score = calculate_obligation_coverage_score(passages, answers)
    print(f"Entailment Score: {entailment_score}")
    print(f"Contradiction Score: {contradiction_score}")
    print(f"Obligation Coverage Score: {obligation_coverage_score}")


    # New formula: (O + E - C + 1) / 3
    composite_score = (obligation_coverage_score + entailment_score - contradiction_score + 1) / 3
    print(f"Final Composite Score: {composite_score}")
    return np.round(composite_score, 5)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/283M [00:00<?, ?B/s]

In [5]:
def main(input_file_path , group_methodName):
    with open(input_file_path, 'r') as file:
        test_data = json.load(file)

    # Prepare the data
    composite_scores = []
    entailment_scores = []
    contradiction_scores = []
    obligation_coverage_scores = []
    total_items = len(test_data)

    for index, item in enumerate(test_data):

        # Merge "RetrievedPassages" if it's a list
        if isinstance(item['RetrievedPassages'], list):
            item['RetrievedPassages'] = " ".join(item['RetrievedPassages'])


        question = [item['QuestionID']]
        passages = [item['RetrievedPassages']]
        answers = [item['Answer']]
        print(f"Processing {index + 1}/{total_items}: QuestionID {question}")

        # Calculate and store scores
        passage_sentences = [sent for passage in passages for sent in sent_tokenize(passage)]
        answer_sentences = [sent for answer in answers for sent in sent_tokenize(answer)]
        entailment_matrix, contradiction_matrix = get_nli_matrix(passage_sentences, answer_sentences)
        entailment_score = calculate_scores_from_matrix(entailment_matrix, 'entailment')
        contradiction_score = calculate_scores_from_matrix(contradiction_matrix, 'contradiction')
        obligation_coverage_score = calculate_obligation_coverage_score(passages, answers)
        final_composite_score = (obligation_coverage_score + entailment_score - contradiction_score + 1) / 3

        # Append to respective lists
        entailment_scores.append(entailment_score)
        contradiction_scores.append(contradiction_score)
        obligation_coverage_scores.append(obligation_coverage_score)
        composite_scores.append(final_composite_score)

    # Calculate averages
    avg_entailment = np.mean(entailment_scores)
    avg_contradiction = np.mean(contradiction_scores)
    avg_obligation_coverage = np.mean(obligation_coverage_scores)
    avg_composite = np.mean(composite_scores)

    # Print and save results to a file
    results = (
        f"Average Entailment Score: {avg_entailment}\n"
        f"Average Contradiction Score: {avg_contradiction}\n"
        f"Average Obligation Coverage Score: {avg_obligation_coverage}\n"
        f"Average Final Composite Score: {avg_composite}\n"
    )
    print(group_methodName)
    print(results)

    # Save the results to a text file
    output_file_path = f"{group_methodName}Results.txt"
    with open(output_file_path, 'w') as output_file:
        output_file.write(results)




In [ ]:
if __name__ == "__main__":
  group_methodName = 'only_bm25_answers'
  main('/content/retrieval_results.passage_only_bm25_answers.json', group_methodName)

Processing 1/446: QuestionID ['f10933cb-8e29-4f37-82ab-0ea23af317ac']
103 passages and 26 answers.


100%|██████████| 103/103 [00:08<00:00, 12.38it/s]
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing 2/446: QuestionID ['c62ccb81-1712-4b11-8a73-26dcbe710272']
3 passages and 26 answers.


100%|██████████| 3/3 [00:00<00:00, 13.87it/s]


Processing 3/446: QuestionID ['5abfd03e-38d5-4428-a594-e323098cd7d9']
1 passages and 22 answers.


...

100%|██████████| 70/70 [00:04<00:00, 14.70it/s]


Processing 442/446: QuestionID ['7897b253-8c88-4832-b7ca-98b10d58a47c']
3 passages and 13 answers.


100%|██████████| 3/3 [00:00<00:00, 22.59it/s]


Processing 443/446: QuestionID ['8e64074a-43c0-4a21-bd2a-790f467f2229']
82 passages and 30 answers.


100%|██████████| 82/82 [00:07<00:00, 10.59it/s]


In [ ]:
if __name__ == "__main__":
  group_methodName = 'rank_fusion_bm25'
  main('/content/retrieval_results.rank_fusion_bm25_answers.json', group_methodName)


Processing 1/446: QuestionID ['f10933cb-8e29-4f37-82ab-0ea23af317ac']
103 passages and 26 answers.


100%|██████████| 103/103 [00:08<00:00, 11.54it/s]


Processing 2/446: QuestionID ['c62ccb81-1712-4b11-8a73-26dcbe710272']
3 passages and 14 answers.


100%|██████████| 3/3 [00:00<00:00, 23.84it/s]


Processing 3/446: QuestionID ['5abfd03e-38d5-4428-a594-e323098cd7d9']
1 passages and 11 answers.


...

100%|██████████| 8/8 [00:00<00:00, 16.86it/s]


Processing 412/446: QuestionID ['1ef8bc22-300d-4f63-aa26-44b07ff51f6f']
4 passages and 10 answers.


100%|██████████| 4/4 [00:00<00:00, 29.65it/s]


Processing 413/446: QuestionID ['560e5c7d-7c38-4c98-9664-025b52e5b5a5']
89 passages and 23 answers.


100%|██████████| 89/89 [00:07<00:00, 12.47it/s]


In [ ]:
if __name__ == "__main__":
  group_methodName = 'golden'
  main('/content/GoldenQuestionAnswerforEvaluationMetric.json', group_methodName)

Processing 1/21: QuestionID ['1']
3 passages and 4 answers.


100%|██████████| 3/3 [00:00<00:00, 27.37it/s]


Processing 2/21: QuestionID ['2']
6 passages and 7 answers.


100%|██████████| 6/6 [00:00<00:00, 24.33it/s]


Processing 3/21: QuestionID ['3']
11 passages and 12 answers.


...

Processing 20/21: QuestionID ['24']
11 passages and 17 answers.


100%|██████████| 11/11 [00:02<00:00,  4.39it/s]


Processing 21/21: QuestionID ['25']
3 passages and 4 answers.


100%|██████████| 3/3 [00:00<00:00, 38.79it/s]


golden
Average Entailment Score: 0.8372866666666667
Average Contradiction Score: 0.2598147619047619
Average Obligation Coverage Score: 1.0
Average Final Composite Score: 0.8591573015873015



In [ ]:
if __name__ == "__main__":
  group_methodName = 'selected'
  main('/content/selected_test_samples.json', group_methodName)

Processing 1/100: QuestionID ['480fBQ']
2 passages and 3 answers.


100%|██████████| 2/2 [00:00<00:00,  7.48it/s]


Processing 2/100: QuestionID ['412Gsb']
2 passages and 2 answers.


100%|██████████| 2/2 [00:00<00:00, 42.63it/s]


Processing 3/100: QuestionID ['498QLc']
2 passages and 8 answers.


...

Processing 99/100: QuestionID ['516QYq']
1 passages and 2 answers.


100%|██████████| 1/1 [00:00<00:00, 43.71it/s]


Processing 100/100: QuestionID ['395PNP']
2 passages and 5 answers.


100%|██████████| 2/2 [00:00<00:00, 37.48it/s]


selected
Average Entailment Score: 0.7024035
Average Contradiction Score: 0.18315340000000002
Average Obligation Coverage Score: 0.34450000000000003
Average Final Composite Score: 0.6212500333333333

